In [1]:
import pickle
import spacy
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "textcat", "ner"])

In [7]:
df = pd.read_csv('data/df_w_r.csv', index_col = 0)

In [8]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery', 'type'],
      dtype='object')

In [9]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,type
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,white
1,US,"Tart and snappy, the flavors of lime flesh and...",Pinot Gris,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,white
3,France,This dry and restrained wine offers spice in p...,Gewürztraminer,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,white
4,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,white


In [10]:
def custom_tokenizer(text):
    '''
    used to filter out unwanted words, punctuation, and so on
    '''
    tokens = []
    for t in nlp(text):
        if not(len(t) < 2 or t.is_stop or t.like_num or 
               t.is_punct or not t.is_alpha):
            tokens.append(t.lemma_)
    return tokens 

In [11]:
corpus = df['description']

In [12]:
corpus

0        Aromas include tropical fruit, broom, brimston...
1        Tart and snappy, the flavors of lime flesh and...
2        Pineapple rind, lemon pith and orange blossom ...
3        This dry and restrained wine offers spice in p...
4        Savory dried thyme notes accent sunnier flavor...
                               ...                        
36504    This is a barely pink rosé, perfect to reflect...
36505    Soft and ripe, it is gently fruity with light ...
36506    Close to the beach of Pampelonne, this is a an...
36507    The use of 100% Syrah gives a deep pink color ...
36508    Picturesque with a crystalline salmon color, i...
Name: description, Length: 36509, dtype: object

In [13]:
y = df['variety']

In [14]:
y.value_counts()

Chardonnay                    11753
Riesling                       5189
Sauvignon Blanc                4967
Rosé                           3564
White Blend                    2360
Pinot Gris                     1455
Grüner Veltliner               1345
Portuguese White               1159
Bordeaux-style White Blend     1066
Pinot Grigio                   1052
Gewürztraminer                 1012
Viognier                        996
Chenin Blanc                    591
Name: variety, dtype: int64

In [15]:
bow = CountVectorizer(tokenizer=custom_tokenizer, 
                      ngram_range=(1, 1), 
                      min_df=0.01, 
                      max_df=0.99)

In [16]:
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, 
                                                              test_size=0.4, 
                                                              train_size=0.6)

In [17]:
X_train = bow.fit_transform(corpus_train)

In [18]:
X_train.shape

(21905, 363)

In [19]:
X_test = bow.transform(corpus_test)

In [20]:
m_rf = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('RFC', RandomForestClassifier())
])

In [21]:
cross_val_score(m_rf, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

0.7055009023724019

In [22]:
m_rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('TfIdf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('RFC',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=

In [23]:
m_rf.predict(X_test)

array(['Chardonnay', 'Bordeaux-style White Blend', 'Chardonnay', ...,
       'Chardonnay', 'White Blend', 'Sauvignon Blanc'], dtype=object)

In [24]:
metrics.accuracy_score(y_test, m_rf.predict(X_test))

0.7201451657080252

In [25]:
m_rf.predict(bow.transform(['cherry flavor nice oak accent']))

array(['Rosé'], dtype=object)

In [26]:
m_rf.predict_proba(bow.transform(['cherry flavor nice oak accent'])).max()

0.48

In [27]:
with open('m_w_r_rf.p', 'wb') as f:
    pickle.dump(m_rf, f)

In [28]:
with open('bow_w_r_rf.p', 'wb') as f:
    pickle.dump(bow, f)